<a href="https://colab.research.google.com/github/justina-tran/yelp-reviews/blob/master/topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyLDAvis

     |████████████████████████████████| 1.7 MB 33.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [10]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
#vis
import pyLDAvis
import pyLDAvis.gensim_models

In [5]:
import os
os.chdir('/content/drive/MyDrive/Projects/NLP/data')

In [8]:
df = pd.read_csv('reviews_cleaned')

In [15]:
df.dropna(inplace=True)

In [16]:
# top words for each sentiment
pos_words = df[df['sentiment'] == 'positive']['cleaned_text']
neg_words = df[df['sentiment'] == 'negative']['cleaned_text']
all_words = pd.concat([pos_words, neg_words])
print(all_words.values.shape)

#create array of pos and neg labels
labels = np.append(np.ones((len(pos_words), 1)), np.zeros((len(neg_words), 1)), axis=0)
print(labels.shape)

(13059,)
(13059, 1)


In [17]:
all_words[0][:90]

'absolutely one favorite ice cream place city recommend swirl black tea/oolong large mini b'

In [18]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(all_words)

print(data_words[0][0:20])

['absolutely', 'one', 'favorite', 'ice', 'cream', 'place', 'city', 'recommend', 'swirl', 'black', 'tea', 'oolong', 'large', 'mini', 'black', 'definitely', 'strong', 'flavor', 'almost', 'coffee']


In [19]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print(corpus[0][0:20])

word = id2word[[0][:1][0]]
print(word)

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]
absolutely


In [20]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.150459  0.360501       1        1  63.644002
0     -0.095648 -0.346108       2        1  10.575683
4     -0.352123 -0.019654       3        1  10.574786
1      0.297580 -0.203626       4        1   8.148025
2      0.300650  0.208888       5        1   7.057504, topic_info=        Term         Freq        Total Category  logprob  loglift
88     order  6251.000000  6251.000000  Default  30.0000  30.0000
19       ice  3724.000000  3724.000000  Default  29.0000  29.0000
2616   ramen  3335.000000  3335.000000  Default  28.0000  28.0000
10     cream  2420.000000  2420.000000  Default  27.0000  27.0000
1416    food  2346.000000  2346.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
811     fill   281.653145   451.408992   Topic5  -4.9249   2.1794
408      eat   356.002348  1286.616612   Topic5  -4.6907   1.3662
15    flavor   407.874137  2649.134821   Topic5  -4.5547   0.7800
453      top   228.824353   628.624791   Topic5  -5.1327   1.6405
203      cup   193.594763  1284.777896   Topic5  -5.2999   0.7585

[231 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
5614      2  0.997124    acai
9452      1  0.058328  ajisen
9452      2  0.053328  ajisen
9452      3  0.639941  ajisen
9452      4  0.228312  ajisen
...     ...       ...     ...
323       1  0.615672    work
323       2  0.383650    work
67        1  0.999750   would
956       3  0.998066    year
1244      3  0.995302    yelp

[247 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 2, 3])